In [1]:
import re

def pattern_matching(output_text):
    """
    Extracts correctness and reasoning score from output_text using regex.
    Handles both markdown-style (bold `**text:**`) and plain text formats.
    """
    # Regex patterns to capture numbers and words after "Reasoning Score:" and "Correctness:"
    reasoning_pattern = r'(?:\*\*Reasoning Score:\*\*|Reasoning Score:)\s*(\d+)'
    correctness_pattern = r'(?:\*\*Correctness:\*\*|Correctness:)\s*(\w+)'

    # Search for Reasoning Score
    score_match = re.search(reasoning_pattern, output_text)
    reasoning_score = int(score_match.group(1)) if score_match else 0  # Return 0 instead of NaN

    # Search for Correctness
    correctness_match = re.search(correctness_pattern, output_text)
    correctness = correctness_match.group(1) if correctness_match else "unknown"  # Default to "unknown" if missing

    return correctness, reasoning_score

In [2]:
import re
import glob
import pandas as pd
from tqdm import tqdm
import math

all_csv = 'all_results_reasoning.csv'
results_df = pd.DataFrame(columns=['csv_path', 'len(df)', 'correctness_acc', 'easy_acc', 'med_acc', 'hard_acc', 'avg_reasoning_score', 'avg_easy_reasoning_score', 'avg_med_reasoning_score', 'avg_hard_reasoning_score'])

csv_paths = glob.glob("*.csv")
for csv_path in tqdm(csv_paths):
    if csv_path == all_csv:
        continue

    easy_corr_list, med_corr_list, hard_corr_list = [], [], []
    easy_reas_list, med_reas_list, hard_reas_list = [], [], []

    correctness_list = []
    reasoning_scores = []

    df = pd.read_csv(csv_path)
    for idx, row in df.iterrows():
        difficulty = row['difficulty']
        correctness = row['correctness']
        reasoning_score = row['reasoning_score']
        output_text = row['eval_result']
        # print(f'{output_text}')

        # if reasoning_score is None or math.isnan(reasoning_score):
        correctness, reasoning_score = pattern_matching(output_text)
        correctness_list.append(correctness)
        reasoning_scores.append(reasoning_score)
        if difficulty == 'Easy':
            easy_corr_list.append(correctness)
            easy_reas_list.append(reasoning_score)
        elif difficulty == 'Medium':
            med_corr_list.append(correctness)
            med_reas_list.append(reasoning_score)
        elif difficulty == 'Hard':
            hard_corr_list.append(correctness)
            hard_reas_list.append(reasoning_score)

        df.at[idx, 'correctness'] = correctness
        df.at[idx, 'reasoning_score'] = reasoning_score
        df.to_csv(csv_path)

        # print(f'correctness: {correctness}, {type(correctness)}')
        # print(f'reasoning_score: {reasoning_score}, {type(reasoning_score)}')
    
    correctness_acc = (correctness_list.count('correct') / len(correctness_list)) * 100
    easy_acc = (easy_corr_list.count('correct') / len(easy_corr_list)) * 100
    med_acc = (med_corr_list.count('correct') / len(med_corr_list)) * 100
    hard_acc = (hard_corr_list.count('correct') / len(hard_corr_list)) * 100

    avg_reasoning_score = (sum(reasoning_scores) / len(reasoning_scores))
    avg_easy_reasoning_score = (sum(easy_reas_list) / len(easy_reas_list))
    avg_med_reasoning_score = (sum(med_reas_list) / len(med_reas_list))
    avg_hard_reasoning_score = (sum(hard_reas_list) / len(hard_reas_list))


    print('\n')
    print("*"*50)
    print(f"csv_path: {csv_path}")
    print(f"len(df): {len(df)}")
    print(f"correctness_acc: {correctness_acc}")
    print(f"easy_acc: {easy_acc}")
    print(f"med_acc: {med_acc}")
    print(f"hard_acc: {hard_acc}")
    print('\n')
    print(f"avg_reasoning_score: {avg_reasoning_score}")
    print(f"avg_easy_reasoning_score: {avg_easy_reasoning_score}")
    print(f"avg_med_reasoning_score: {avg_med_reasoning_score}")
    print(f"avg_hard_reasoning_score: {avg_hard_reasoning_score}")
    print("*"*50)
    print('\n')

    new_row = {
        "csv_path": csv_path,
        "len(df)": len(df),
        "correctness_acc": correctness_acc,
        "easy_acc": easy_acc,
        "med_acc": med_acc,
        "hard_acc": hard_acc,
        "avg_reasoning_score": avg_reasoning_score,
        "avg_easy_reasoning_score": avg_easy_reasoning_score,
        "avg_med_reasoning_score": avg_med_reasoning_score,
        "avg_hard_reasoning_score": avg_hard_reasoning_score,
    }

    new_idx = len(results_df)
    results_df.loc[new_idx] = new_row
    results_df.to_csv(all_csv)


  8%|▊         | 2/25 [00:32<06:08, 16.00s/it]



**************************************************
csv_path: target_deepseek_14b_judge_llama_3_1_8B_it_attack_alter.csv
len(df): 193
correctness_acc: 96.37305699481865
easy_acc: 98.33333333333333
med_acc: 97.32142857142857
hard_acc: 85.71428571428571


avg_reasoning_score: 4.694300518134715
avg_easy_reasoning_score: 4.633333333333334
avg_med_reasoning_score: 4.741071428571429
avg_hard_reasoning_score: 4.619047619047619
**************************************************




 12%|█▏        | 3/25 [00:48<05:52, 16.04s/it]



**************************************************
csv_path: target_deepseek_14b_judge_llama_3_1_8B_it_attack_irrelevant.csv
len(df): 235
correctness_acc: 100.0
easy_acc: 100.0
med_acc: 100.0
hard_acc: 100.0


avg_reasoning_score: 4.965957446808511
avg_easy_reasoning_score: 4.955223880597015
avg_med_reasoning_score: 4.978260869565218
avg_hard_reasoning_score: 4.931034482758621
**************************************************




 16%|█▌        | 4/25 [01:10<06:28, 18.50s/it]

...



**************************************************
csv_path: target_qwq_32b_judge_llama_3_1_8B_it_attack_irrelevant.csv
len(df): 291
correctness_acc: 97.59450171821305
easy_acc: 98.57142857142858
med_acc: 97.0059880239521
hard_acc: 98.11320754716981


avg_reasoning_score: 4.962199312714777
avg_easy_reasoning_score: 4.957142857142857
avg_med_reasoning_score: 4.970059880239521
avg_hard_reasoning_score: 4.943396226415095
**************************************************




 96%|█████████▌| 24/25 [04:55<00:07,  7.94s/it]



**************************************************
csv_path: target_qwq_32b_judge_llama_3_1_8B_it_attack_naive.csv
len(df): 298
correctness_acc: 97.6510067114094
easy_acc: 98.57142857142858
med_acc: 96.51162790697676
hard_acc: 100.0


avg_reasoning_score: 4.939597315436242
avg_easy_reasoning_score: 4.9
avg_med_reasoning_score: 4.965116279069767
avg_hard_reasoning_score: 4.909090909090909
**************************************************




100%|██████████| 25/25 [05:09<00:00, 12.38s/it]



**************************************************
csv_path: target_qwq_32b_judge_llama_3_1_8B_it_attack_paraphrase.csv
len(df): 296
correctness_acc: 99.32432432432432
easy_acc: 98.57142857142858
med_acc: 100.0
hard_acc: 98.14814814814815


avg_reasoning_score: 4.949324324324325
avg_easy_reasoning_score: 4.942857142857143
avg_med_reasoning_score: 4.941520467836257
avg_hard_reasoning_score: 4.981481481481482
**************************************************


